In [76]:
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

half = 0 # determines what half the game is in --> first half = 0

# What web page do we want? 
pageAddress = "https://www.cbssports.com/nba/gametracker/live/NBA_20240429_OKC@NO/"
statsAddress="https://www.teamrankings.com/ncaa-basketball/stat/2nd-half-points-per-game"
# Open and read in the web page 
soup = BeautifulSoup(urlopen(pageAddress), "html.parser")
soup2=BeautifulSoup(urlopen(statsAddress), "html.parser")
table2=soup2.find('table')
df2 = pd.read_html(str(table2))[0]
print(df2)
team1_stats=df2[df2['Team']=='Clemson']
team1_scor_avg=team1_stats['2023']
team1_scor_avg=team1_scor_avg[team1_scor_avg.rfind(" "):]
team2_scor_avg=df2[df2['Team']=='Arizona']
print(team1_scor_avg)
print(team2_scor_avg)

     Rank         Team  2023  Last 3  Last 1  Home  Away  2022
0       1      Alabama  46.8    43.0    32.0  47.1  46.6  43.4
1       2     Kentucky  45.4    46.0    41.0  46.6  43.8  38.6
2       3      Arizona  44.3    41.0    41.0  49.0  40.6  40.9
3       4     Lipscomb  43.9    46.0    32.0  48.5  40.8  39.6
4       5      Samford  43.8    45.7    51.0  46.9  40.9  40.2
..    ...          ...   ...     ...     ...   ...   ...   ...
357   358    Loyola-MD  32.1    26.3    21.0  29.8  33.5  34.6
358   359         Army  32.0    33.3    41.0  32.4  31.6  34.8
359   360    Coppin St  31.3    27.3    26.0  32.5  30.7  36.8
360   361    Dartmouth  31.2    34.7    38.0  32.5  30.2  35.3
361   362  Miss Val St  28.9    36.0    39.0  33.7  26.9  32.3

[362 rows x 8 columns]


AttributeError: 'Series' object has no attribute 'rfind'

In [77]:
# Read the table of data into a Dataframe 
table=soup.find_all('table')
#tables = section.find('div',class_="ResponsiveTable playByPlay__table playByPlay__table--default")


df = pd.read_html(str(table))[half]
# Filter rows where 'Score' is not 'NaN'
filtered_df = df.dropna(subset=['Score'])

# Get the last score from the filtered DataFrame
last_score_row = filtered_df.iloc[0]
last_score=last_score_row['Score']
last_time=last_score_row['Time']
home_score=last_score[0:last_score.find('-')]
away_score=last_score[last_score.find('-')+1:]
print(home_score)
print(away_score)
print(last_time)
print("Last Score row:", last_score)
num_tables = len(table)
print(num_tables)

96
89
0:36
Last Score row: 96-89
38


In [64]:
# Find scoring rates for each team
#last_time = "18:21"
home_score = int(home_score)
away_score = int(away_score)

home_score = 65
away_score =58

minutes, seconds = map(int, last_time.split(':'))
#time_rem = minutes * 60 + seconds
time_rem = 5*60
print(time_rem)
half=1
if(half == 0):
    if(time_rem!=20*60):
        time_played = 20 * 60 - time_rem
    time_rem=time_rem+20*60
else:
    time_played = 20 * 60 * 2 - time_rem
if(time_rem > 0):                      
    home_rate = home_score / (time_played)
    away_rate = away_score / (time_played)
else:
    home_rate = 0
    away_rate = 0

print(time_rem)
print(time_played)

300
300
2100


In [65]:
home_wins = 0
away_wins = 0
total_home_score = 0
total_away_score = 0
print(home_rate, away_rate)
#home_rate = .4
#away_rate = .3
#print(home_rate, away_rate)
games = 0
if(time_rem!=40*60):
    for i in range(10000):
        time = time_rem
        h_score = home_score
        a_score = away_score

        while (time_rem > 0):
            # Simulate time until next scoring event for both teams
            home_time = np.random.exponential(1 / home_rate)
            away_time = np.random.exponential(1 / away_rate)

             # Determine which team scores next
            if (home_time < away_time):
                h_score += 1
            else:
                a_score += 1

            # Update remaining time
            time -= min(home_time, away_time)

            # Check if game is over
            if (time <= 0 ):
                break
        games+= 1
        if (h_score > a_score):
            home_wins += 1
        elif(h_score < a_score):
            away_wins += 1
        else:
            if(home_time > away_time):
                home_wins +=1
            else:
                away_wins +=1
        total_home_score += h_score
        total_away_score += a_score
        #print(h_score, "-" , a_score)

    home_win_percent =  home_wins/games 
    print(home_win_percent*100)
    total_home_score = total_home_score / games
    total_away_score = total_away_score / games
    print(total_home_score, " home score")
    print(total_away_score, " away score")

#print(games)

0.030952380952380953 0.02761904761904762
97.11
74.7854  home score
66.7884  away score


In [66]:
# taking percentage from the colley and adding it
%store -r winProbability
print( winProbability*100)

factor=1
if time_rem<40*60 and time_rem>=30*60:
    factor=0.8
elif time_rem<30*60 and time_rem>20*60:
    factor=0.6
elif time_rem==20*60:
    factor=0.5
elif time_rem<20*60 and time_rem>=10*60:
    factor=0.4
elif time_rem<10*60:
    factor=0.2
print(factor)


final_prob=factor*(winProbability)+(1-factor)*(home_win_percent)
print(final_prob*100)

11.874697491136942
0.2
80.0629394982274
